# Stochastic Heat Equation Benchmarks

In this notebook we will benchmark against the stochastic heat equation with Dirichlet BCs and scalar noise. The function for generating the problem is as follows:

Stochastic Heat Equation with scalar multiplicative noise

S-ROCK: CHEBYSHEV METHODS FOR STIFF STOCHASTIC
DIFFERENTIAL EQUATIONS

ASSYR ABDULLE AND STEPHANE CIRILLI

Raising D or k increases stiffness

In [168]:
using StochasticDiffEq, DiffEqNoiseProcess

function generate_stiff_stoch_heat(D=1,k=1;N = 100, t_end = 3.0, adaptivealg = :RSwM3)
    A = full(Tridiagonal([1.0 for i in 1:N-1],[-2.0 for i in 1:N],[1.0 for i in 1:N-1]))
    dx = 1/N
    A = D/(dx^2) * A
    function f(t,u,du)
        A_mul_B!(du,A,u)
    end
    #=
    function f(::Type{Val{:analytic}},t,u0,W)
        expm((A-k/2)*t+W*I)*u0 # no -k/2 for Strat
    end
    =#
    function g(t,u,du)
        @. du = k*u
    end
    SDEProblem(f,g,ones(N),(0.0,t_end),noise=WienerProcess(0.0,0.0,0.0,rswm=RSWM(adaptivealg=adaptivealg)))
end

N = 100
D = 1; k = 1
    A = full(Tridiagonal([1.0 for i in 1:N-1],[-2.0 for i in 1:N],[1.0 for i in 1:N-1]))
    dx = 1/N
    A = D/(dx^2) * A;

Now lets solve it with high accuracy.

In [5]:
prob = generate_stiff_stoch_heat(1.0,1.0)
@time sol = solve(prob,SRIW1(),progress=true,abstol=1e-6,reltol=1e-6);

 12.166893 seconds (5.21 M allocations: 587.500 MiB, 1.75% gc time)


## Highest dt

Let's try to find the highest possible dt:

In [10]:
@time sol = solve(generate_stiff_stoch_heat(1.0,1.0),SRIW1());

  2.541107 seconds (1.14 M allocations: 105.601 MiB)


In [156]:
@time sol = solve(generate_stiff_stoch_heat(1.0,1.0),SRIW1(),progress=true,adaptive=false,dt=0.00005);

  2.064195 seconds (934.67 k allocations: 80.410 MiB, 11.75% gc time)


In [155]:
@time sol = solve(generate_stiff_stoch_heat(1.0,1.0),EM(),progress=true,adaptive=false,dt=0.00005);

  1.398688 seconds (850.56 k allocations: 77.130 MiB, 1.64% gc time)


In [154]:
@time sol = solve(generate_stiff_stoch_heat(1.0,1.0),ImplicitRKMil(),progress=true,dt=0.1);

  0.020710 seconds (1.21 k allocations: 458.375 KiB)


In [153]:
@time sol = solve(generate_stiff_stoch_heat(1.0,1.0),ImplicitRKMil(),progress=true,dt=0.01);

  0.109937 seconds (4.15 k allocations: 798.438 KiB)


In [157]:
@time sol = solve(generate_stiff_stoch_heat(1.0,1.0),ImplicitRKMil(),progress=true,dt=0.001);

  2.559465 seconds (42.80 k allocations: 5.466 MiB)


In [159]:
@time sol = solve(generate_stiff_stoch_heat(1.0,1.0),ImplicitEM(),progress=true,dt=0.001);

  2.547884 seconds (42.79 k allocations: 5.464 MiB)


## Simple Error Analysis
Now let's check the error at an arbitrary timepoint in there. Our analytical solution only exists in the Stratanovich sense, so we are limited in the methods we can calculate errors for.

In [138]:
function simple_error(alg;kwargs...)
    sol = solve(generate_stiff_stoch_heat(1.0,1.0,t_end=0.25),alg;kwargs...);
    sum(abs2,sol[end] - expm(A*sol.t[end]+sol.W[end]*I)*prob.u0)
end

simple_error (generic function with 1 method)

In [139]:
mean(simple_error(EulerHeun(),dt=0.00005) for i in 1:400)

3.3466532629323837e-9

In [141]:
mean(simple_error(ImplicitRKMil(interpretation=:Stratanovich),dt=0.1) for i in 1:400)

5.432339032712796

In [144]:
mean(simple_error(ImplicitRKMil(interpretation=:Stratanovich),dt=0.01) for i in 1:400)

0.18019725952244847

In [160]:
mean(simple_error(ImplicitRKMil(interpretation=:Stratanovich),dt=0.001) for i in 1:400)

0.01615085247634074

In [163]:
mean(simple_error(ImplicitEulerHeun(),dt=0.001) for i in 1:400)

6.865143745355041e-6

In [165]:
mean(simple_error(ImplicitEulerHeun(),dt=0.01) for i in 1:400)

0.16644294279128125

In [166]:
mean(simple_error(ImplicitEulerHeun(),dt=0.1) for i in 1:400)

5.364797907695661

## Interesting Property

Note that RSwM1 and RSwM2 are not stable on this problem.

In [169]:
sol = solve(generate_stiff_stoch_heat(1.0,1.0,adaptivealg=:RSwM1),SRIW1());

## Conclusion

In this problem, the implicit methods do not have a stepsize limit. This is because the stiffness almost entirely deteriministic due to diffusion. In that case, if we do not care about the error too much, the implicit methods dominate. Of course, as the tolerance gets lower there is a tradeoff point where the higher order methods will become more efficient. The explicit methods are clearly stability-bound and thus unless we want an error of like 10^-10 we are better off using an implicit method here.